In [1]:
%pylab inline

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import argparse
import os
from collections import defaultdict

import torch

from habitat.config import Config as CN
from habitat.utils.visualizations.utils import images_to_video, observations_to_image

from habitat_baselines.common.baseline_registry import baseline_registry
from habitat_baselines.common.environments import get_env_class, NavRLEnv
from habitat_baselines.config.default import get_config
from habitat_baselines.utils.common import batch_obs, generate_video
from habitat_baselines.utils.env_utils import construct_envs

from my_habitat_baselines.resnet_policy import PointNavResNetPolicy

In [3]:
jupyter_dir = "/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter"

jupyter_dir

'/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter'

In [4]:
#
# specify args
#

model_path = "/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models"

arg_string = ""

arg_string += "--model-path %s" % model_path

arg_string += \
"""
--sensors RGB_SENSOR
--hidden-size 512
--normalize-visual-inputs 1
--backbone resnet50
--num-recurrent-layers 2
TEST_EPISODE_COUNT 5
TASK_CONFIG.SIMULATOR.NOISE_MODEL.CONTROLLER Proportional
TASK_CONFIG.SIMULATOR.NOISE_MODEL.NOISE_MULTIPLIER 0.5
TASK_CONFIG.SIMULATOR.RGB_SENSOR.HFOV 45
TASK_CONFIG.SIMULATOR.DEPTH_SENSOR.HFOV 45
TASK_CONFIG.SIMULATOR.TURN_ANGLE 30
TASK_CONFIG.SIMULATOR.AGENT_0.RADIUS 0.20
TASK_CONFIG.DATASET.DATA_PATH obstacle_1/{split}/{split}.json.gz
TASK_CONFIG.DATASET.SPLIT minival
TASK_CONFIG.ENVIRONMENT.GENERATE_ON_FLY False
TASK_CONFIG.SIMULATOR.RGB_SENSOR.POSITION [0,0.6096,0]
TASK_CONFIG.SIMULATOR.DEPTH_SENSOR.POSITION [0,0.6096,0]
VIDEO_OPTION ['disk']
TASK_CONFIG.TASK.TOP_DOWN_MAP.MAP_RESOLUTION 5000
"""

print(arg_string)

parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, required=True)
parser.add_argument("--sensors", type=str, required=True)
parser.add_argument("--hidden-size", type=int, required=True)
parser.add_argument(
    "--normalize-visual-inputs", type=int, required=True, choices=[0, 1]
)
parser.add_argument(
    "--backbone",
    type=str,
    required=True,
    choices=["resnet50", "se_resneXt50"],
)
parser.add_argument("--num-recurrent-layers", type=int, required=True)
parser.add_argument(
    "opts",
    default=None,
    nargs=argparse.REMAINDER,
    help="Modify config options from command line",
)
args = parser.parse_args(arg_string.split())

print(args)

--model-path /Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models
--sensors RGB_SENSOR
--hidden-size 512
--normalize-visual-inputs 1
--backbone resnet50
--num-recurrent-layers 2
TEST_EPISODE_COUNT 5
TASK_CONFIG.SIMULATOR.NOISE_MODEL.CONTROLLER Proportional
TASK_CONFIG.SIMULATOR.NOISE_MODEL.NOISE_MULTIPLIER 0.5
TASK_CONFIG.SIMULATOR.RGB_SENSOR.HFOV 45
TASK_CONFIG.SIMULATOR.DEPTH_SENSOR.HFOV 45
TASK_CONFIG.SIMULATOR.TURN_ANGLE 30
TASK_CONFIG.SIMULATOR.AGENT_0.RADIUS 0.20
TASK_CONFIG.DATASET.DATA_PATH obstacle_1/{split}/{split}.json.gz
TASK_CONFIG.DATASET.SPLIT minival
TASK_CONFIG.ENVIRONMENT.GENERATE_ON_FLY False
TASK_CONFIG.SIMULATOR.RGB_SENSOR.POSITION [0,0.6096,0]
TASK_CONFIG.SIMULATOR.DEPTH_SENSOR.POSITION [0,0.6096,0]
VIDEO_OPTION ['disk']
TASK_CONFIG.TASK.TOP_DOWN_MAP.MAP_RESOLUTION 5000

Namespace(backbone='resnet50', hidden_size=512, model_path='/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models', normalize_visual_inputs=1, num_re

In [5]:
#
# load and customize config
#

habitat_dir = "/Users/mroberts/code/github/habitat-lab"
os.chdir(habitat_dir)

config = get_config(
    "habitat_baselines/config/pointnav/ppo_pointnav.yaml"
)

config.defrost()
config.TASK_CONFIG.SIMULATOR.NOISE_MODEL = CN()
config.TASK_CONFIG.SIMULATOR.NOISE_MODEL.CONTROLLER = None
config.TASK_CONFIG.SIMULATOR.NOISE_MODEL.NOISE_MULTIPLIER = None
config.TASK_CONFIG.SIMULATOR.RGB_SENSOR.HFOV = None
config.TASK_CONFIG.SIMULATOR.DEPTH_SENSOR.HFOV = None
config.TASK_CONFIG.ENVIRONMENT.GENERATE_ON_FLY = None
config.freeze()

config.merge_from_list(args.opts)

# config.defrost()
# config.TASK_CONFIG.SIMULATOR.ACTION_SPACE_CONFIG = "pyrobotnoisy"
# config.freeze()

# config.defrost()
# config.TASK_CONFIG.SIMULATOR.NOISE_MODEL.ROBOT = "LoCoBot"
# config.TASK_CONFIG.SIMULATOR.NOISE_MODEL.CONTROLLER = "ILQR"    # our pre-trained model lists "proportional" in the filename, so don't change to ILQR 
# config.TASK_CONFIG.SIMULATOR.NOISE_MODEL.NOISE_MULTIPLIER = 1.0 # our pre-trained model lists "0.5" in the filename, so don't change to 1.0
# config.freeze()

config.defrost()
config.TASK_CONFIG.ENVIRONMENT.ITERATOR_OPTIONS.SHUFFLE = False
config.freeze()

config.defrost()
config.TASK_CONFIG.SIMULATOR.HABITAT_SIM_V0.ALLOW_SLIDING = False
config.freeze()

config.defrost()
config.TASK_CONFIG.DATASET.CONTENT_SCENES = ["*"]
config.TASK_CONFIG.DATASET.DATA_PATH = "data/datasets/pointnav/gibson/v2/val/val.json.gz" # don't have obstacle_1 scenes, so use Gibson instead
config.freeze()

config.defrost()
config.NUM_ENVIRONMENTS = 1
config.freeze()

config.defrost()
if args.sensors == "":
    config.SENSORS = []
else:
    config.SENSORS = args.sensors.split(",")
# TODO(akadian): collisions are not working
# config.TASK_CONFIG.TASK.MEASUREMENTS.append("COLLISIONS")
config.freeze()

print(config)

BASE_TASK_CONFIG_PATH: configs/tasks/pointnav_gibson.yaml
CHECKPOINT_FOLDER: data/new_checkpoints
CHECKPOINT_INTERVAL: -1
CMD_TRAILING_OPTS: []
ENV_NAME: NavRLEnv
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/new_checkpoints
FORCE_BLIND_POLICY: False
FORCE_TORCH_SINGLE_THREADED: True
LOG_FILE: train.log
LOG_INTERVAL: 25
NUM_CHECKPOINTS: 100
NUM_ENVIRONMENTS: 1
NUM_PROCESSES: -1
NUM_UPDATES: 10000
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambased/data/ORBvoc.txt
PROFILING:
  CAPTURE_START_STEP: -1
  NUM_STEPS_TO_CAPT

In [6]:
#
# create device
#

device = (
    torch.device("cuda:{}".format(config.TORCH_GPU_ID))
    if torch.cuda.is_available()
    else torch.device("cpu")
)

device

device(type='cpu')

In [7]:
#
# construct a single env instead of multiple envs for simplicity
#

env = NavRLEnv(config)

2021-11-17 20:04:32,473 Initializing dataset PointNav-v1
2021-11-17 20:04:32,493 initializing sim Sim-v0
I1117 20:04:36.979046 12177 simulator.py:221] Loaded navmesh data/scene_datasets/gibson/Cantwell.navmesh
I1117 20:04:32.496129 63790528 ManagedFileBasedContainer.h:210] <Dataset>::convertFilenameToPassedExt : Filename : default changed to proposed scene_dataset_config.json filename : default.scene_dataset_config.json
I1117 20:04:32.496166 63790528 AttributesManagerBase.h:365] <Dataset>::createFromJsonOrDefaultInternal : Proposing JSON name : default.scene_dataset_config.json from original name : default | This file  does not exist.
I1117 20:04:32.496249 63790528 AssetAttributesManager.cpp:120] Asset attributes (capsule3DSolid : capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
I1117 20:04:32.496290 63790528 AssetAttributesManager.cpp:120] Asset attributes (capsule3DWireframe : capsule3DWireframe_hemiRings_8_c

Renderer: AMD Radeon Pro 5500M OpenGL Engine by ATI Technologies Inc.
OpenGL version: 4.1 ATI-3.10.22
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_texture_storage
    GL_EXT_texture_filter_anisotropic
    GL_EXT_debug_label
    GL_EXT_debug_marker
Using driver workarounds:
    no-layout-qualifiers-on-old-glsl
    apple-buffer-texture-unbind-on-buffer-modify


ts/gibson/Cantwell.glb.
I1117 20:04:36.978076 63790528 SceneDatasetAttributes.cpp:45] ::addNewSceneInstanceToDataset : Dataset : 'default' : Stage Attributes 'data/scene_datasets/gibson/Cantwell.glb' specified in Scene Attributes exists in dataset library.
I1117 20:04:36.978080 63790528 SceneDatasetAttributes.cpp:85] ::addNewSceneInstanceToDataset : Dataset : 'default' : Lighting Layout Attributes no_lights specified in Scene Attributes exists in dataset library.
I1117 20:04:36.978092 63790528 Simulator.cpp:171] ::reconfigure : createSceneInstance success == true for active scene name : data/scene_datasets/gibson/Cantwell.glb with renderer.
I1117 20:04:36.992566 63790528 PathFinder.cpp:382] Building navmesh with 306x145 cells
I1117 20:04:37.082011 63790528 PathFinder.cpp:652] Created navmesh with 199 vertices 94 polygons
I1117 20:04:37.082381 63790528 Simulator.cpp:790] reconstruct navmesh successful


In [8]:
#
# get starting episode and scene so we can reset the environment's episode iterator
#

initial_episode_id = env.current_episode.episode_id
initial_scene_id = env.current_episode.scene_id

print(initial_scene_id, initial_episode_id)

observation = env.reset()

print(env.current_episode.scene_id, env.current_episode.episode_id)

data/scene_datasets/gibson/Cantwell.glb 0
data/scene_datasets/gibson/Cantwell.glb 0


In [9]:
#
# load model
#

def load_model(
    path,
    observation_space,
    action_space,
    hidden_size,
    normalize_visual_inputs,
    backbone,
    num_recurrent_layers,
    device,
):

    model = PointNavResNetPolicy(
        observation_space=observation_space,
        action_space=action_space,
        hidden_size=hidden_size,
        normalize_visual_inputs=normalize_visual_inputs,
        backbone=backbone,
        num_recurrent_layers=num_recurrent_layers,
        goal_sensor_uuid="pointgoal_with_gps_compass",
    )

    model.to(device)

    saved_model = torch.load(path, map_location=device)
    saved_model_state_dict = {}
    for k, v in saved_model["state_dict"].items():
        new_k = k.replace("actor_critic.", "")
        saved_model_state_dict[new_k] = v

    model.load_state_dict(saved_model_state_dict)

    model_params = 0
    for k,v in model.state_dict().items():
        # print(k, torch.numel(v))
        model_params += torch.numel(v)
    print(model_params)

    saved_model_params = 0
    for k,v in saved_model["state_dict"].items():
        # print(k, torch.numel(v))
        saved_model_params += torch.numel(v)
    print(saved_model_params)

    return model

In [12]:
def eval_model(model_path, num_episodes=-1, max_num_actions_per_episode=10000):

    global observation
    
    print("Resetting env to initial_episode_id and initial_scene_id...")
    print()

    while env.current_episode.episode_id != initial_episode_id or env.current_episode.scene_id != initial_scene_id:
        observation = env.reset()
        
    print()
    print(env.current_episode.scene_id, env.current_episode.episode_id)
    print()
    
    #
    # load model
    #
    
    model = load_model(
        path=model_path,
        observation_space=env.observation_space,
        action_space=env.action_space,
        hidden_size=args.hidden_size,
        normalize_visual_inputs=bool(args.normalize_visual_inputs),
        backbone=args.backbone,
        num_recurrent_layers=args.num_recurrent_layers,
        device=device,
    )

    model.eval()

    #
    # initialization before main loop
    #

    metric_name = config.TASK_CONFIG.TASK.MEASUREMENTS[0]
    metric_cfg = getattr(config.TASK_CONFIG.TASK, metric_name)
    measure_type = baseline_registry.get_measure(metric_cfg.TYPE)
    metric_uuid = measure_type(None, None)._get_uuid()

    assert measure_type is not None, "invalid measurement type {}".format(metric_cfg.TYPE)

    print(metric_name)
    print(metric_cfg)
    print(measure_type)
    print(metric_uuid)
    print()

    print(len(env.episodes))
    print()

    print(env.current_episode)
    print()

    print(env._env.get_metrics())
    print()

    observations = [observation]
    batch = batch_obs(observations, device)

    num_processes = 1

    test_recurrent_hidden_states = torch.zeros(
        model.net.num_recurrent_layers,
        num_processes,
        args.hidden_size,
        device=device,
    )
    prev_actions = torch.zeros(
        num_processes, 1, device=device, dtype=torch.long
    )
    not_done_masks = torch.zeros(num_processes, 1, device=device)
    print(not_done_masks)

    current_episode_num_actions = 0
    current_episode_reward = 0.0
    current_episode_stats_actions = defaultdict(int)

    stats_episodes = dict()  # dict of dicts that stores stats per episode

    #
    # main loop
    #

    max_num_actions_per_episode = 10000
    
    if num_episodes == -1:
        num_episodes = len(env.episodes)

    while len(stats_episodes) < num_episodes:

        #
        # main loop: choose action
        #

        with torch.no_grad():
            _, actions, _, test_recurrent_hidden_states = model.act(
                batch,
                test_recurrent_hidden_states,
                prev_actions,
                not_done_masks,
                deterministic=False,
            )

            prev_actions.copy_(actions)

        assert len(actions) == 1
        action = actions[0]

        # print(env.habitat_env.task.get_action_name(action.item()))

        #
        # main loop: perform action
        #

        observation, reward, done, info = env.step(action=action.item())

        #
        # main loop: update state
        #

        observations = [observation]
        batch = batch_obs(observations, device)

        dones = [done]
        not_done_masks = torch.tensor(
            [[0.0] if done else [1.0] for done in dones],
            dtype=torch.float,
            device=device,
        )

        current_episode_num_actions += 1
        current_episode_reward += reward
        current_episode_stats_actions[action.item()] += 1

        assert current_episode_num_actions < max_num_actions_per_episode

        if done:

            # record stats
            stats_episode = dict(info)
            stats_episode["reward"] = current_episode_reward
            stats_episode["stats_actions"] = dict(current_episode_stats_actions)

            # if len(stats_episodes) % 100 == 0:
            #     print("Episodes finished: {}".format(len(stats_episodes)))

            print("Episodes finished: {}".format(len(stats_episodes)))
            print(stats_episode)
            print()
            
            stats_episodes[ (env.current_episode.scene_id, env.current_episode.episode_id) ] = stats_episode

            # reset env
            observation = env.reset()
            observations = [observation]
            batch = batch_obs(observations, device)

            test_recurrent_hidden_states = torch.zeros(
                model.net.num_recurrent_layers,
                num_processes,
                args.hidden_size,
                device=device,
            )
            prev_actions = torch.zeros(
                num_processes, 1, device=device, dtype=torch.long
            )
            not_done_masks = torch.zeros(num_processes, 1, device=device)

            current_episode_num_actions = 0
            current_episode_reward = 0.0
            current_episode_stats_actions = defaultdict(int)
            
    return stats_episodes

In [45]:
# models_to_eval = {}

model_name = "rgb_train_sliding_off_noise_0.0"
model_path = \
"/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models/" + \
"job_19633792"                        + "." \
"sensor_RGB_SENSOR"                   + "." \
"train_data_gibson"                   + "." \
"noise_multiplier_0.0"                + "." \
"noise_model_controller_Proportional" + "." \
"agent_radius_0.20"                   + "." \
"success_reward_10.0"                 + "." \
"slack_reward_-0.01"                  + "." \
"collision_reward_0.0"                + "." \
"spl_max_collisions_500"              + "_" \
"ckpt.000000049"                      + \
".pth"
models_to_eval[model_name] = {"model_path" : model_path}

# model_name = "rgb_train_sliding_off_noise_0.5"
# model_path = \
# "/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models/" + \
# "job_19633792"                        + "." \
# "sensor_RGB_SENSOR"                   + "." \
# "train_data_gibson"                   + "." \
# "noise_multiplier_0.5"                + "." \
# "noise_model_controller_Proportional" + "." \
# "agent_radius_0.20"                   + "." \
# "success_reward_10.0"                 + "." \
# "slack_reward_-0.01"                  + "." \
# "collision_reward_0.0"                + "." \
# "spl_max_collisions_500"              + "_" \
# "ckpt.000000049"                      + \
# ".pth"
# models_to_eval[model_name] = {"model_path" : model_path}

model_name = "rgb_train_sliding_off_noise_1.0"
model_path = \
"/Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models/" + \
"job_19633792"                        + "." \
"sensor_RGB_SENSOR"                   + "." \
"train_data_gibson"                   + "." \
"noise_multiplier_1.0"                + "." \
"noise_model_controller_Proportional" + "." \
"agent_radius_0.20"                   + "." \
"success_reward_10.0"                 + "." \
"slack_reward_-0.01"                  + "." \
"collision_reward_0.0"                + "." \
"spl_max_collisions_500"              + "_" \
"ckpt.000000049"                      + \
".pth"
models_to_eval[model_name] = {"model_path" : model_path}

for m in models_to_eval.items():
    if "stats_episodes" not in m[1].keys():
        print("Evaluating model: " + m[0])
        print("")
        m[1]["stats_episodes"] = eval_model(m[1]["model_path"])
    else:
        print("Skipping model: " + m[0])
        print("")

print("Finished.")

Skipping model: rgb_train_sliding_off_noise_0.5

Evaluating model: rgb_train_sliding_off_noise_0.0

Resetting env to initial_episode_id and initial_scene_id...



IndexError: list index out of range

In [73]:
# import pickle

# pickle_file = os.path.join(jupyter_dir, "models_to_eval.pickle")

# print("Saving to " + pickle_file)
# print()

# with open(pickle_file, "wb") as p:
#     pickle.dump(models_to_eval, p)

# print("Finished.")
# print()

Saving to /Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models_to_eval.pickle

Finished.



In [76]:
import pickle

pickle_file = os.path.join(jupyter_dir, "models_to_eval.pickle")

print("Loading from " + pickle_file)
print()

with open(pickle_file, "rb") as p:
    models_to_eval = pickle.load(p)

print("Finished.")
print()

Loading from /Users/mroberts/code/github/interiorsim/code/experiments/srcc/jupyter/models_to_eval.pickle

Finished.



In [77]:
stats_episodes = models_to_eval["rgb_train_sliding_off_noise_0.5"]["stats_episodes"]

valid = np.array([ np.isfinite(s[1]["distance_to_goal"]) for s in stats_episodes.items() ])

print(np.count_nonzero(valid) / valid.shape[0])
print(np.count_nonzero(valid))
print(valid.shape[0])
    
success = np.array([ s[1]["success"] for s in stats_episodes.items() ])
spl = np.array([ s[1]["spl"] for s in stats_episodes.items() ])

success = success[valid]
spl = spl[valid]

print(np.mean(success))
print(np.mean(spl))

0.9688128772635815
963
994
0.5244029075804777
0.34438900960606356
